In [ ]:
import pandas as pd
import gc
import numpy as np
from sklearn.preprocessing import LabelEncoder
from scipy.stats import linregress

study_name = "lightgbm6"

df = pd.read_csv("../datasets/sell-in.txt.gz", sep="\t")
productos = pd.read_csv("../datasets/tb_productos.txt", sep="\t")
productos = productos.drop_duplicates(subset=["product_id"],keep="first")  # Eliminar duplicados por si acaso
stocks = pd.read_csv("../datasets/tb_stocks.txt", sep="\t")

productos_ok = pd.read_csv("https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/product_id_apredecir201912.txt", sep="\t")
df = df[df["product_id"].isin(productos_ok["product_id"])]
df = df.groupby(by=["periodo","product_id"]).agg({"tn":"sum","plan_precios_cuidados":"first"}).reset_index()


productos_list = df['product_id'].unique()
periodos = df['periodo'].unique()
idx = pd.MultiIndex.from_product([productos_list, periodos], names=['product_id', 'periodo'])
completo = idx.to_frame(index=False)
# 4 filtrar combinaciones periodo_producto
completo = completo.merge(df, on=['periodo', 'product_id'], how='left')
nacimiento_producto = df.groupby('product_id')['periodo'].min().rename('nacimiento_producto')

completo = completo.merge(nacimiento_producto, on='product_id', how='left')
completo = completo[completo['periodo'] >= completo['nacimiento_producto']]



df = pd.merge(completo, productos, how="left", on="product_id")
df = df.merge(stocks, how="left", on=["product_id", "periodo"])

df["periodo"] = pd.to_datetime(df["periodo"], format="%Y%m")

df["mes"] = df["periodo"].dt.month
df["year"] = df["periodo"].dt.year
df["quarter"] = df["periodo"].dt.quarter
# shift t-1 a t12
meses = ["tn"]
for i in range(1, 13):
    df[f"tn_t{i}"] = df.groupby("product_id")["tn"].shift(i)
    df[f"delta_t{i}"] = df[f"tn"] / df[f"tn_t{i}"]
    
    
    meses.append(f"tn_t{i}")
df["promedio_t12"] = df[meses].mean(axis=1)
df["promedio_t6"] = df[meses[:6]].mean(axis=1)

# Función para calcular la pendiente de una serie temporal
def calcular_pendiente(serie):
    datos = serie.dropna().values
    if len(datos) < 2:  # Mínimo 2 puntos para una regresión
        return np.nan
    x = np.arange(len(datos))  # [0, 1, 2, ..., n-1] (representa el tiempo)
    slope, _, _, _, _ = linregress(x, datos)
    return slope

# Aplicamos la función por grupo (producto) y reindexamos para alinear con df
df["pendiente_reg_12"] = (
    df.groupby("product_id")[meses]
    .apply(lambda grupo: grupo.apply(calcular_pendiente, axis=1))
    .reset_index(level=0, drop=True)  # Eliminamos el nivel de grupo para alinear índices
)

# totales por cat1 por periodo
df["cat1_total"] = df.groupby(["periodo", "cat1"])["tn"].transform("sum")
# totales por cat2 por periodo
df["cat2_total"] = df.groupby(["periodo", "cat2"])["tn"].transform("sum")
# totales por cat3 por periodo
df["cat3_total"] = df.groupby(["periodo", "cat3"])["tn"].transform("sum")

# target t+2
df["target"] = df.groupby("product_id")["tn"].shift(-2)




cat_cols = ['cat1', 'cat2', 'cat3', 'brand', 'plan_precios_cuidados']
for col in cat_cols:
    if col in df.columns:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str))
        
del completo, productos, stocks
gc.collect()
df

,product_id,periodo,tn,plan_precios_cuidados,nacimiento_producto,cat1,cat2,cat3,brand,sku_size,...,delta_t11,tn_t12,delta_t12,promedio_t12,promedio_t6,pendiente_reg_12,cat1_total,cat2_total,cat3_total,target
0,20001,2017-01-01,934.77222,0,201701,1,10,47,0,3000,...,NaN,NaN,NaN,934.772220,934.772220,NaN,16043.73199,8020.78937,2883.78193,1303.35771
1,20001,2017-02-01,798.01620,0,201701,1,10,47,0,3000,...,NaN,NaN,NaN,866.394210,866.394210,136.756020,16020.70298,7852.24082,2841.64107,1069.96130
2,20001,2017-03-01,1303.35771,0,201701,1,10,47,0,3000,...,NaN,NaN,NaN,1012.048710,1012.048710,-184.292745,23236.56968,11332.00333,3971.28823,1502.20132
3,20001,2017-04-01,1069.96130,0,201701,1,10,47,0,3000,...,NaN,NaN,NaN,1026.526858,1026.526858,-91.090875,19548.34338,9046.89989,3104.20199,1520.06539
4,20001,2017-05-01,1502.20132,0,201701,1,10,47,0,3000,...,NaN,NaN,NaN,1121.661750,1121.661750,-140.680330,23489.30687,12213.39157,4223.53944,1030.67391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22370,21087,2019-12-01,1.02205,0,201909,2,3,3,23,65,...,NaN,NaN,NaN,0.907423,0.907423,-0.265787,5106.38727,961.24517,616.94575,NaN
22371,21214,2019-09-01,0.34250,0,201909,2,3,65,23,50,...,NaN,NaN,NaN,0.342500,0.342500,NaN,6539.06554,998.16591,28.14845,0.84012
22372,21214,2019-10-01,0.21735,0,201909,2,3,65,23,50,...,NaN,NaN,NaN,0.279925,0.279925,0.125150,7643.88143,1223.87956,41.89931,0.24428
22373,21214,2019-11-01,0.84012,0,201909,2,3,65,23,50,...,NaN,NaN,NaN,0.466657,0.466657,-0.248810,7345.75572,1082.10256,29.63557,NaN


In [2]:
# Primero asegurémonos que ambas columnas son datetime
df['nacimiento_producto'] = pd.to_datetime(df['nacimiento_producto'], format="%Y%m")
df['periodo'] = pd.to_datetime(df['periodo'])

# Ahora calculamos la diferencia en meses
df['meses_desde_lanzamiento'] = ((df['periodo'].dt.year - df['nacimiento_producto'].dt.year) * 12 + 
                                 (df['periodo'].dt.month - df['nacimiento_producto'].dt.month))

df['nacimiento_producto'] = df['nacimiento_producto'].astype(int)

In [3]:
# Estacionalidad anual (comparación con mismo mes año anterior)
for i in [12, 24]:
    df[f'tn_anterior_{i}meses'] = df.groupby('product_id')['tn'].shift(i)
    df[f'ratio_anual_{i}meses'] = df['tn'] / df[f'tn_anterior_{i}meses']

In [4]:
# Media móvil de 3 y 6 meses
df['media_movil_3m'] = df.groupby('product_id')['tn'].transform(lambda x: x.rolling(3).mean())
df['media_movil_6m'] = df.groupby('product_id')['tn'].transform(lambda x: x.rolling(6).mean())

In [5]:
# Porcentaje de ventas del producto en su categoría
df['pct_cat1'] = df['tn'] / df['cat1_total']
df['pct_cat2'] = df['tn'] / df['cat2_total']
df['pct_cat3'] = df['tn'] / df['cat3_total']

In [6]:
# Coeficiente de variación de las ventas
df['cv_producto'] = df.groupby('product_id')['tn'].transform('std') / df.groupby('product_id')['tn'].transform('mean')

In [7]:
# Diferencia entre medias móviles corta y larga
df['diff_media_3_6'] = df['media_movil_3m'] - df['media_movil_6m']

In [8]:
# Percentiles de ventas por producto
df['percentil_25_producto'] = df.groupby('product_id')['tn'].transform(lambda x: x.quantile(0.25))
df['percentil_75_producto'] = df.groupby('product_id')['tn'].transform(lambda x: x.quantile(0.75))

In [9]:
# Meses de temporada alta (puedes ajustar según tu dominio)
df['temporada_alta'] = df['mes'].isin([11, 12, 1, 2]).astype(int)

In [10]:
# Interacción entre mes y categoría
df['mes_cat1'] = df['mes'] * df['cat1']

In [11]:
# Promedio ponderado de lags recientes
df['lag_ponderado'] = (df['tn_t1']*0.5 + df['tn_t2']*0.3 + df['tn_t3']*0.2)

In [12]:
# Cambio porcentual acumulado en últimos 3 meses
df['acum_change_3m'] = (df['tn'] - df['tn_t3']) / df['tn_t3']

In [13]:
# Promedio histórico por mes para cada producto
df['media_mensual_producto'] = df.groupby(['product_id', 'mes'])['tn'].transform('mean')

In [14]:
# Cambio de dirección en la tendencia
df['cambio_direccion'] = ((df['tn'] > df['tn_t1']) & (df['tn_t1'] < df['tn_t2'])).astype(int)

In [15]:
# Estadísticas por marca
df['media_marca'] = df.groupby(['periodo', 'brand'])['tn'].transform('mean')

In [16]:
# Diferencia con el mismo mes del año anterior
df['diff_estacional'] = df['tn'] - df['tn_anterior_12meses']

In [17]:
# Momentum de ventas (diferencia entre últimos 3 meses y anteriores 3 meses)
df['momentum_3m'] = (df['tn_t1'] + df['tn_t2'] + df['tn_t3']) - (df['tn_t4'] + df['tn_t5'] + df['tn_t6'])

In [18]:
df_kgl = df[df["periodo"] == "2019-12-01"]
df_kgl["periodo"] = df_kgl["periodo"].astype(int)
df = df[~df["periodo"].isin(["2019-11-01", "2019-12-01"])]


C:\Users\carre\AppData\Local\Temp\ipykernel_16748\2505419454.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_kgl["periodo"] = df_kgl["periodo"].astype(int)
C:\Users\carre\AppData\Local\Temp\ipykernel_16748\2505419454.py:3: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  df = df[~df["periodo"].isin(["2019-11-01", "2019-12-01"])]


In [19]:

# Separar features y target
X = df.drop(columns=["target"])
y = df["target"]
X_train = X[X["periodo"] < '2019-10-01']
X_train["periodo"] = X_train["periodo"].astype(int)
y_train = y[X["periodo"] < '2019-10-01']

X_val = X[X["periodo"] == '2019-10-01']
X_val["periodo"] = X_val["periodo"].astype(int)
y_val = y[X["periodo"] == '2019-10-01']



C:\Users\carre\AppData\Local\Temp\ipykernel_16748\4176657755.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train["periodo"] = X_train["periodo"].astype(int)
C:\Users\carre\AppData\Local\Temp\ipykernel_16748\4176657755.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val["periodo"] = X_val["periodo"].astype(int)


In [20]:
import lightgbm as lgb
import optuna
from sklearn.metrics import mean_squared_error, mean_absolute_error
import os

# === 2. Definición del objetivo para Optuna ===
def objective(trial):
    params = {
        "sample_weight": X_train["tn"].to_list(),
        "objective": "regression",
        "metric": ["rmse", "mape"],
        "boosting_type": "gbdt",
        "verbosity": -1,
        "n_jobs": -1,
        "seed": 42,
        "num_leaves": trial.suggest_int("num_leaves", 20, 200),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.2),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.6, 0.9),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.7, 0.95),
        "bagging_freq": trial.suggest_int("bagging_freq", 0, 5),
        "min_child_samples": trial.suggest_int("min_child_samples", 10, 50),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-3, 5.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-3, 5.0, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "max_bin": trial.suggest_int("max_bin", 10, 2000),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 200),
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        "path_smooth": trial.suggest_float("path_smooth", 0, 1),  # Nuevo parámetro útil
    }

    model = lgb.train(
        params,
        lgb.Dataset(X_train, label=y_train),
        valid_sets=[lgb.Dataset(X_val, label=y_val)],
        num_boost_round=1000,
        callbacks=[lgb.early_stopping(stopping_rounds=100, verbose=False)]
    )

    preds = model.predict(X_val)
    rmse = mean_squared_error(y_val, preds)
    rmse = np.sqrt(rmse)
    
    #rmse = mean_squared_error(y_val, preds, squared=False)
    return rmse

# === 3. Configurar almacenamiento SQLite para Optuna ===
os.makedirs("optuna_storage", exist_ok=True)
DB_PATH = "optuna_storage/optuna.db"
STUDY_NAME = study_name
storage_url = f"sqlite:///{DB_PATH}"

# === 4. Crear o cargar estudio ===
study = optuna.create_study(
    study_name=STUDY_NAME,
    storage=storage_url,
    direction="minimize",
    load_if_exists=True
)

# === 5. Ejecutar optimización ===
study.optimize(objective, n_trials=100)

# === 6. Mostrar resultados ===
print("Mejores hiperparámetros encontrados:")
print(study.best_params)
print(f"Mejor MAE: {study.best_value:.4f}")

c:\Users\carre\anaconda3\envs\py311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-07-12 01:28:58,675] A new study created in RDB with name: lightgbm6
[I 2025-07-12 01:29:01,392] Trial 0 finished with value: 21.76451850406012 and parameters: {'num_leaves': 106, 'learning_rate': 0.19436933959398095, 'feature_fraction': 0.7456976469652901, 'bagging_fraction': 0.780820462105736, 'bagging_freq': 0, 'min_child_samples': 28, 'lambda_l1': 0.008355837891260952, 'lambda_l2': 0.10074269591743233, 'max_depth': 5, 'max_bin': 1703, 'min_data_in_leaf': 79, 'extra_trees': False, 'path_smooth': 0.4407832484126537}. Best is trial 0 with value: 21.76451850406012.
[I 2025-07-12 01:29:04,488] Trial 1 finished with value: 30.671906202798123 and parameters: {'num_leaves': 114, 'learning_rate': 0.09345717471922176, 'feature_fra

Mejores hiperparámetros encontrados:
{'num_leaves': 193, 'learning_rate': 0.15065872231574762, 'feature_fraction': 0.8725129205383041, 'bagging_fraction': 0.9329598177376311, 'bagging_freq': 1, 'min_child_samples': 34, 'lambda_l1': 0.0024190174865305436, 'lambda_l2': 0.01745012698759562, 'max_depth': 7, 'max_bin': 1745, 'min_data_in_leaf': 20, 'extra_trees': False, 'path_smooth': 0.90998766364336}
Mejor MAE: 19.3484


In [24]:
best_params = study.best_params.copy()
best_params.update({
    "objective": "regression",
    "metric": ["rmse", "mape"],
    "verbosity": -1,
    "n_jobs": -1,
    "seed": 42,
    "sample_weight": X_train["tn"].to_list(),
})

model = lgb.train(
    best_params,
    lgb.Dataset(X_train, label=y_train),
    valid_sets=[lgb.Dataset(X_val, label=y_val)],
    num_boost_round=1000,
    callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=False)]
)

# Asegurar las mismas columnas
X_kgl = df_kgl[X_train.columns]  # Misma estructura

# === 9. Hacer predicción sobre nuevos datos ===
preds_kgl = model.predict(X_kgl)


In [25]:
result = pd.DataFrame({"product_id": X_kgl["product_id"],  "tn": preds_kgl})

result = result[result["product_id"].isin(productos_ok["product_id"])]

result

,product_id,tn
35,20001,1483.635866
71,20002,1163.438858
107,20003,771.281878
143,20004,499.830891
179,20005,527.439673
...,...,...
22358,20962,5.914705
22362,20975,5.598132
22366,20995,5.598132
22370,21087,4.384427


In [26]:
result.to_csv(f"{study_name}.csv",sep=',', index=False)